# Exercises

Let's test ourselves on the concepts covered in this lesson. Consider the following two tables from a database we're making to keep track of our vinyl LP collection. Start by reviewing these `CREATE TABLE` statements:

```
CREATE TABLE albums (
    album_id bigint GENERATED ALWAYS AS IDENTITY,
    catalog_code text,
    title text,
    artist text,
    release_date date,
    genre text,
    description text
);

CREATE TABLE songs (
    song_id bigint GENERATED ALWAYS AS IDENTITY,
    title text,
    composers text,
    album_id bigint
);
```

The `albums` table includes information specific to the overall collection of songs on the disc. The `songs` table catalogs each track on the album. Each song has a title & a column for its composers, who might be different than the album artist.

Use the tables to answer these questions:

1. Modify these `CREATE TABLE` statements to include primary & foreign keys plus additional constraints on both tables. Explain why you made your choices.
2. Instead of using `album_id` as a surrogate key for your primary key, are there any columns in `albums` that could be useful as a natural key? What would you have to know to decide?
3. To speed up queries, which columns are good candidates for indexes?

---

# 1.

An album needs songs, but songs don't need albums -- artists can put out a single. So, I think it would be great if I added a constraint to the `albums` table that references the `songs` table to check if there is a matching `album_id`. I also think that since if an artist puts out an album, he/she would be be the composer of the songs in that album. However, there also can be colabs between artists for certain songs. This means that there can be multiple artists who composed a single song. This means that we need also need to add a constraint to the `albums` table that references its `artist` column with the `composers` column of the `songs` table, checking to see if the artist's name appears in the `composers` column. 

```
CREATE TABLE songs (
    song_id bigint GENERATED ALWAYS AS IDENTITY,
    title text NOT NULL,
    composers text NOT NULL,
    album_id bigint
);

CREATE TABLE albums (
    album_id bigint GENERATED ALWAYS AS IDENTITY
        REFERENCES songs (album_id),
    catalog_code text,
    title text NOT NULL,
    artist text
        REFERENCES songs (composers),
    release_date date NOT NULL,
    genre text NOT NULL,
    description text
);
```

The `REFERENCES` constraints were added (explained above). Unfortunately, `CHECK` constraints don't work in other tables. I'm sure there is a way to write custom SQL functions, but I don't know how to do that yet. Because of potential for collaboration, I was hoping I could check that the artist name appears in the `composers` column of the `songs` table, using some sort of text detection logical expression. I didn't use any `UNIQUE` constraints, because I feel like there is always the possibility of artists naming their songs the same thing, or naming their albums the same thing. Albums could drop on the same day. There are only so many genres. I used a lot of `NOT NULL` constraints. For the `songs` table, based on what I see on Youtube, there is always a name of the song & the artist who put it out. For the `albums` table, similar logic, there is always a title for an album & an artist associated with that album. Usually, artists put out songs all of a certain genre, but on the off chance that they do create an album in a genre that hasn't existed before, people are quick to put a name to it. I didn't know what `catalog_code` meant, so I left it as is. I also left `description` as is, in case the album needs some explanation.

# 2.

Since I don't know what `catalog_code` is I'll leave that one alone. I do think we can create a natural key using a combination of the `title`, `artist`, & `release_date` columns. Maybe even `title` & `artist` is enough -- I find it hard to believe that any individual artist would drop two albums with the same name on the same day.

So, if we pretended the `album_id` didn't exist, we could create a natural key using the three columns I just mentioned:

```
CREATE TABLE albums (
    catalog_code text,
    title text,
    artist text,
    release_date date,
    genre text,
    description text,
    CONSTRAINT album_key PRIMARY KEY (
        title, artist, release_date
    )
);
```

# 3.

For the `songs` table, `title` & `composers` would be great candidates for indexes, because they can quickly narrow down the song or related album you're looking for. For the `albums` table, you can narrow down the list of albums quickly if you index the `artist` or `genre` columns. I feel as though the stated columns could be used in a table join with each other, or could end up in a `WHERE` query, which are some of the considerations when using indexes stated in this lesson.